# The Python concurrency story

notes
- Let's begin with what concurrency means
- Web search led me to this blog with the same name, I'll leave a link there. Check it out, it has useful benchmarks etc.

## Concurrency

[dictionary.com](http://www.dictionary.com/browse/concurrent)
- occurring or existing simultaneously or side by side
- acting in conjunction; cooperating

notes
- Ok, that's the literal dictionary definition, but why is it relevant for us as people writing software, and why does the world seem to care suddenly ?

### Waiters and tables
- 5 tables
- 1 waiter: who can take one order at a time
- Customers(Table) think about the order for 4 seconds
- Waiter takes down order in 1 seconds


In [10]:
# sequential example
import time
from datetime import datetime

class Waiter(object):

    def __init__(self, name):
        self.name = name
    
    def take_order(self):
        time.sleep(1)

class Table(object):

    def __init__(self, name):
        self.order_taken = False
        self.name = name
    
    def give_order(self, waiter):
        print(f"{self.name} is thinking about the order")
        time.sleep(4)
        self.order_taken = True
        waiter.take_order()
        print(f"{self.name} has given it's order to {waiter.name}")

tables = [Table("A"), Table("B"), Table("C"), Table("D"), Table("E")]
waiter = Waiter("John Doe")

start_time = datetime.now()
for table in tables:
    table.give_order(waiter)
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"taking all orders took {elapsed_time.seconds} seconds")


A is thinking about the order
A has given it's order to John Doe
B is thinking about the order
B has given it's order to John Doe
C is thinking about the order
C has given it's order to John Doe
D is thinking about the order
D has given it's order to John Doe
E is thinking about the order
E has given it's order to John Doe
taking all orders took 25 seconds


### The free lunch is over
- approaching the physical limits of Moore's Law, processors not getting faster
- way forward is to have MORE processors, not faster processors
- Ahmdal's Law says that gains are bounded by how much of the program has to run in a sequential manner. (The more you can can parallelize, the faster you can go)
- allows for scaling 'horizontally'
- explains the sudden surge of interest in Erlang, Scala, Clojure and FP

[The free lunch is over](http://www.gotw.ca/publications/concurrency-ddj.htm)

notes:
- moore's law says that the number of transistors on an IC doubles every two years, which directly correlates with processor speed
- corresponds to scaling the system vertically, i.e increasing the throughput of a system
- on the other hand scaling horizontally means adding more instances of the same system to increase throughput
- these are the languages which offer better constructs for concurrency.

notes:

- The phenomena we're talking about are everpresent around us, though they're of particular interest in CS.
- If all this talk of processors, and web scale confused you, let's take a more grounded example.
- I find it helpful when learning new things to have the same thing presented to me in different ways
- So let's make our restaurant example concurrent. Instead of blocking the waiter while the customer is thinking, the waiter goes to whichever table is ready to order

In [11]:
# multithreaded example
import threading
import time
from datetime import datetime

class Waiter(object):
    lock = threading.Lock()
    def __init__(self, name):
        self.name = name
    
    def take_order(self):
        with self.lock:  # TODO: queue example if adequate time
        # waiter can take only one order at a time.
        # Sleep does not block the entire Python process
        # here context switch will happen to another thread
        # so we have to lock so that the waiter isn't
        # allowed to be at two tables at once
            time.sleep(1)

class Table(object):

    def __init__(self, name):
        self.order_taken = False
        self.name = name
    
    def give_order(self, waiter):
        print(f"{self.name} is thinking about the order")
        time.sleep(4)
        self.order_taken = True
        waiter.take_order()
        print(f"{self.name} has given it's order to {waiter.name}")

tables = [Table("A"), Table("B"), Table("C"), Table("D"), Table("E")]
waiter = Waiter("John Doe")

start_time = datetime.now()

threads = []
for table in tables:
    thread = threading.Thread(target=table.give_order, args=(waiter,))
    # all tables start thinking about ordering
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()  # wait for all threads to finish
    # otherwise execution will continue without waiting for threads
    # to end, and we'll get elapsed time as 0.

end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"took {elapsed_time.seconds} seconds")

A is thinking about the order
B is thinking about the order
C is thinking about the order
D is thinking about the order
E is thinking about the order
B has given it's order to John Doe
A has given it's order to John Doe
E has given it's order to John Doe
D has given it's order to John Doe
C has given it's order to John Doe
took 9 seconds


## Concurrency is not parallelism
Concurrency is not parallelism

concurrency: Dealing with a lot at once, property of the solution (code)
parallelism: Doing a lot at once, property of the runtimeb

### Why you should care
- Knowing this distinction allows you to pick the right tool for the right job
- Apparently, Python is bad for programming concurrent programs because only one python thread can run at a time.

In [9]:
# sequential_client.py
import requests
from datetime import datetime

start_time = datetime.now()
for i in range(5):
    print(requests.get("http://127.0.0.1:5000").text)
end_time = datetime.now()
print(f"elapsed time {(end_time - start_time).seconds} s")

Hello, World!, I am thread Thread-1
Hello, World!, I am thread Thread-2
Hello, World!, I am thread Thread-3
Hello, World!, I am thread Thread-4
Hello, World!, I am thread Thread-5
elapsed time 5 s


In [ ]:
# threaded_client.py
import threading
import datetime

def make_request():
    print(requests.get("http://127.0.0.1:5000").text)

notes
- Here, orders are being processed concurrently, even though there's no parallelism.
- We can add parallelism by adding another waiter
- Let's talk about python
- We used python threads in the second approach, let's talk more about them.

## Threading in python
- User-level threads: Threads that we can actively create, run, and kill for all of our various tasks (Python)
- Kernel-level threads: Very low-level threads acting on behalf of the operating system

__Advantages__
- Multiple threads are excellent for speeding up blocking I/O bound programs
- Schedulted preemptively - don't have to write any extra code for it
- They are lightweight in terms of memory footprint when compared to processes
- Threads share resources, and thus communication between them is easier
__Disdavantages__
- CPython threads are hamstrung by the limitations of the global interpreter lock, which means that python threads can't make use of multiple CPU cores
(GIL)
- While communication between threads may be easier, you must be very careful not to implement code that is subject to race conditions. It is a comparitively quite difficult to get this right.
- Hard to test, hard to spot bugs, hard to reproduce bugs
- It's computationally expensive to switch context between multiple threads. By adding multiple threads, you could see a degradation in your program's overall performance if not used correctly.

notes:
- Preemptive scheduling is a double edged sword like most things.
- Easy to execute threads, but since you can't make any assumptions about when the task switch might happen, you have to guard portions of your code that access a shared resource(The critical section)
- This access is synchronized via locks, but this is notoriously hard to get right.

### The dreaded GIL
- Python has one global lock, on the entire interpreter instead of thousands of granular locks everywhere else.
- Locking and unlocking is not free, previous attempts at removing the GIL has resulted in severe degradation of the performance of a single thread.

Notes:
- Python threads have to acquire a mutex lock on the interpreter to execute.
- What that essentially means is that only one thread can run in a python process at one time, utilizing only one core at a time.
- It is necessary because python interpreters internal datastructures aren't thread safe
- (Book says memory management isn't thread safe - same thing I think). Previous attempts at
- removing the GIL have been unsuccessful as the overhead of locking degraded performance considerably. So CPU bound tasks are going to be much slower, were the gil removed. The model of having GIL with parallelism deferred to the OS via the
- For utilizing multiple cores, multiprocessing module works well.
- Note that the GIL is present in only the default implementation - CPython, and doesn't exist in runtimes who support parallel threads like Jython, IronPython

When to use
- I/O bound processes
- Blocking I/O

### Python's approach to parallelism - Don't DIY
- To make use of all cores, python prescribes multiprocessing.
- Utilizing multiple cores is left up to the OS (since processes are scheduled by the OS).

__Advantages__:
- They are better than multiple threads at handling CPU-intensive tasks
- We can sidestep the limitations of the GIL by spawning multiple processes
- Crashing processes will not kill our entire program(workers model - give example of Gunicorn - If you've run any python webapp/service in production, you might have heard of it. When your program is leaking memory, just reload it. No shared state - you can do it)

__Disadvantages__:
- No shared memory between processes - have to implement some form of IPC, which is much more resource heavy on the computer.
- slower context switches.
- more startup cost.


notes:
- Let's see this in action.
- Starting from our non blocking IO example
(show non blocking IO multi processing example)

## Async
- cooperative scheduling - need explicit code to cause task switch.
- Typically single threaded
- Since you control when task switches occur, you don't need locks for synchronization.
__Advantages__:
- very low cost, since no context switch. Incidentally they're cheaper than function calls. Cheaper switching mechanism among all techniques. People prefer this model to locking, 
- No cost of synchronization = less CPU consumption. Async servers > threaded servers. You can run many many more async tasks than threads.
__disadvantages__:
- Need code that gives up control
- Need an event loop
- Everything has to be non-blocking.
- Need to learn a lot of new things - new syntax, new libraries(aysnc versions), eventloops, futures.

### Back at the restaurant
An order task involves:
- Choosing items
- placing your order with the waiter
- waiting for your order
- Digging in🍴
- Waiter goes around the tables, and finally to the kitchen (scheduler/eventloop)

In [12]:
# async example
import asyncio
import time
from datetime import datetime

class Waiter(object):

    def __init__(self, name):
        self.name = name
    
    def take_order(self):
        # blocks thread, hence blocking the entire event loop.
        time.sleep(1)

class Table(object):

    def __init__(self, name):
        self.order_taken = False
        self.name = name

    # async keyword defines a coroutine
    async def give_order(self, waiter):
        print(f"{self.name} is thinking about the order")
        await asyncio.sleep(4)  # This is how we give up control
        # Execution will resume after 4 s.
        self.order_taken = True
        waiter.take_order()
        print(f"{self.name} has given it's order to {waiter.name}")

tables = [Table("A"), Table("B"), Table("C"), Table("D"), Table("E")]
waiter = Waiter("John Doe")

eventloop = asyncio.get_event_loop()
start_time = datetime.now()
tasks = [eventloop.create_task(table.give_order(waiter))
         for table in tables]
await asyncio.wait(tasks)
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"taking all orders took {elapsed_time.seconds} seconds")


A is thinking about the order
B is thinking about the order
C is thinking about the order
D is thinking about the order
E is thinking about the order
A has given it's order to John Doe
B has given it's order to John Doe
C has given it's order to John Doe
D has given it's order to John Doe
E has given it's order to John Doe
taking all orders took 9 seconds


notes:
- when you're choosing what to order, the waiter need not be blocked.
- similarly, your program doesn't have to block on a slow network call, your program can be doing other things. This is the foundation of AJAX
- Unicode supports emojis now.